In [1]:
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import spacy
import en_core_web_sm
spacy.load('en_core_web_sm')
from transformers import BertTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


$1.Preprocessing$

In [2]:
# Loading the dataset to a data frame
data_df = pd.read_csv('/Users/jinlinchen/Documents/Study/HWR Berlin/Semester 2/Analytics Lab/Analytics Project/Database Part/manual_label - consolidated.csv')

In [3]:
# Check dataframe
data_df

,entity,sentence,paragraph,sentence_original,label,class_ID
0,retailer,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3
1,Proposition 1,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,belongs_to_article,1
2,Supply-Chain,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3
3,Petri net,which can be equivalently represented with a P...,which can be equivalently represented with a P...,which can be equivalently represented with a P...,background_information,3
4,business processes,which are sent by business processes accounts ...,A simple example of a synthesis process for IA...,which are sent by business processes accounts ...,background_information,3
...,...,...,...,...,...,...
248,database,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1
249,100 employees,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1
250,North America,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",background_information,3
251,e-mail,which they evaluate using Enron 's e - mail da...,Overview: Abbasi and Chen's (2008) CyberGate s...,which they evaluate using Enron's e-mail datab...,related_work,2


In [5]:
# Convert all entries in 'sentence_original' to strings
data_df['sentence_original'] = data_df['sentence_original'].astype(str)

# Remove spaces, puctuation and numbers from original senteces, 
# because they do not convey relevant information for text analysis.
data_df['sentence_preprocessed'] = data_df['sentence_original'].apply(strip_numeric)
data_df['sentence_preprocessed'] = data_df['sentence_preprocessed'].apply(strip_punctuation)
data_df['sentence_preprocessed'] = data_df['sentence_preprocessed'].apply(strip_multiple_whitespaces)

# Transform all letters to lower case -->  words with capital letter might confuse the machine.
# Exp: For machine 'What' is not the same as 'what'
data_df['sentence_preprocessed']=data_df['sentence_preprocessed'].str.lower()

In [6]:
data_df

,entity,sentence,paragraph,sentence_original,label,class_ID,sentence_preprocessed
0,retailer,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3,which gives the supplier s profit as and the r...
1,Proposition 1,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,belongs_to_article,1,which gives the supplier s profit as and the r...
2,Supply-Chain,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3,which gives the supplier s profit as and the r...
3,Petri net,which can be equivalently represented with a P...,which can be equivalently represented with a P...,which can be equivalently represented with a P...,background_information,3,which can be equivalently represented with a p...
4,business processes,which are sent by business processes accounts ...,A simple example of a synthesis process for IA...,which are sent by business processes accounts ...,background_information,3,which are sent by business processes accounts ...
...,...,...,...,...,...,...,...
248,database,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1,which was compiled from two different sources ...
249,100 employees,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1,which was compiled from two different sources ...
250,North America,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",background_information,3,which was compiled from two different sources ...
251,e-mail,which they evaluate using Enron 's e - mail da...,Overview: Abbasi and Chen's (2008) CyberGate s...,which they evaluate using Enron's e-mail datab...,related_work,2,which they evaluate using enron s e mail datab...


In [7]:
# Sort stopwords from nltk.
# According to Chatgpt, nltk has a more comprehensive stopword list than Gensim. 
# And entity classification often involves working with a variety of text data where removing irrelevant words is crucial. Thus, we are using nltk.
sorted_nltk_stopwords = sorted(nltk_stopwords)
print("NLTK stopwords:", sorted_nltk_stopwords)



NLTK stopwords: ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't

In [8]:
# Trimming stopword list, as negations are essencial, they could change the meaning of the whole sentence. 
stopwords_to_remove = ["aren't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
                       'needn', "needn't", 'no', 'nor', 'not', "shan't", 'shouldn', "shouldn't",
                       'wasn', "wasn't", "weren't", 'wouldn', "wouldn't"]

for word in stopwords_to_remove:
    if word in nltk_stopwords:
        nltk_stopwords.remove(word)
sorted_nltk_stopwords = sorted(nltk_stopwords)
print("NLTK stopwords:", sorted_nltk_stopwords)

NLTK stopwords: ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', 'she', "she's", 'should', "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'th

In [9]:
# Define the function to filter stopwords from text
def filter_stopwords(text, stopwords):
    words = text.split()
    words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(words)

# Apply the function to filter stopwords
data_df['sentence_preprocessed'] = data_df['sentence_preprocessed'].apply(lambda x: filter_stopwords(x, nltk_stopwords))

# Short words are not removed as we have abbreviations in our text, and that might be useful. Exp: IoT, RFID, AI, ML


In [10]:
# Check
data_df

,entity,sentence,paragraph,sentence_original,label,class_ID,sentence_preprocessed
0,retailer,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3,gives supplier profit retailer profit proposit...
1,Proposition 1,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,belongs_to_article,1,gives supplier profit retailer profit proposit...
2,Supply-Chain,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3,gives supplier profit retailer profit proposit...
3,Petri net,which can be equivalently represented with a P...,which can be equivalently represented with a P...,which can be equivalently represented with a P...,background_information,3,equivalently represented petri net without tra...
4,business processes,which are sent by business processes accounts ...,A simple example of a synthesis process for IA...,which are sent by business processes accounts ...,background_information,3,sent business processes accounts receivable ca...
...,...,...,...,...,...,...,...
248,database,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1,compiled two different sources namely membersh...
249,100 employees,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1,compiled two different sources namely membersh...
250,North America,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",background_information,3,compiled two different sources namely membersh...
251,e-mail,which they evaluate using Enron 's e - mail da...,Overview: Abbasi and Chen's (2008) CyberGate s...,which they evaluate using Enron's e-mail datab...,related_work,2,evaluate using enron e mail database multiple ...


In [11]:
# Stemming: Convert the word to the root form
data_df['sentence_stemmed'] = data_df['sentence_preprocessed'].apply(stem_text)

In [12]:
# Lemmatization:Convert the word to the linguistic root form


#initialize en
en_core_web_sm = en_core_web_sm.load()
# Extracting the lemmas and join (chatgpt)
def extract_lemmas(text):
# Process the text with the model
    doc = en_core_web_sm(text)
# Extract lemmas for each token in the document
    lemmas = [token.lemma_ for token in doc]
# Join the lemmas into a single string
    return " ".join(lemmas)

data_df['sentence_lemmas'] = data_df['sentence_preprocessed'].apply(extract_lemmas)


In [13]:
# Tokenization (BERT): A predefined word set like BERT. Then the text is matched to a token in the word set. If a word is missing, the word will be splitted until there is a match.

#load the Bert tokenizer
bert_uncased = BertTokenizer.from_pretrained('bert-base-uncased')

data_df['sentence_tokenized'] = data_df['sentence_preprocessed'].apply(lambda text: bert_uncased.tokenize(text))

In [14]:
data_df

,entity,sentence,paragraph,sentence_original,label,class_ID,sentence_preprocessed,sentence_stemmed,sentence_lemmas,sentence_tokenized
0,retailer,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3,gives supplier profit retailer profit proposit...,give supplier profit retail profit proposit ne...,give supplier profit retailer profit propositi...,"[gives, supplier, profit, retailer, profit, pr..."
1,Proposition 1,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,belongs_to_article,1,gives supplier profit retailer profit proposit...,give supplier profit retail profit proposit ne...,give supplier profit retailer profit propositi...,"[gives, supplier, profit, retailer, profit, pr..."
2,Supply-Chain,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3,gives supplier profit retailer profit proposit...,give supplier profit retail profit proposit ne...,give supplier profit retailer profit propositi...,"[gives, supplier, profit, retailer, profit, pr..."
3,Petri net,which can be equivalently represented with a P...,which can be equivalently represented with a P...,which can be equivalently represented with a P...,background_information,3,equivalently represented petri net without tra...,equival repres petri net without transit illus...,equivalently represent petri net without trans...,"[equivalent, ##ly, represented, pet, ##ri, net..."
4,business processes,which are sent by business processes accounts ...,A simple example of a synthesis process for IA...,which are sent by business processes accounts ...,background_information,3,sent business processes accounts receivable ca...,sent busi process account receiv case inform r...,send business process account receivable case ...,"[sent, business, processes, accounts, rec, ##e..."
...,...,...,...,...,...,...,...,...,...,...
248,database,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1,compiled two different sources namely membersh...,compil two differ sourc name membership list l...,compile two different source namely membership...,"[compiled, two, different, sources, namely, me..."
249,100 employees,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",belongs_to_article,1,compiled two different sources namely membersh...,compil two differ sourc name membership list l...,compile two different source namely membership...,"[compiled, two, different, sources, namely, me..."
250,North America,which was compiled from two different sources ...,The questionnaire was administered to managers...,"which was compiled from two different sources,...",background_information,3,compiled two different sources namely membersh...,compil two differ sourc name membership list l...,compile two different source namely membership...,"[compiled, two, different, sources, namely, me..."
251,e-mail,which they evaluate using Enron 's e - mail da...,Overview: Abbasi and Chen's (2008) CyberGate s...,which they evaluate using Enron's e-mail datab...,related_work,2,evaluate using enron e mail database multiple ...,evalu us enron e mail databas multipl onlin di...,evaluate use enron e mail database multiple on...,"[evaluate, using, en, ##ron, e, mail, database..."


In [15]:
data_df.to_csv('preprocessed_data(manual).csv', index=False)

Next step see embeddings entity.ipynb and embeddings sentence.ipynb